In [ ]:
# utility stuff
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import ipywidgets as wdg
import numpy as np
from IPython.display import display

from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, show
output_notebook(hide_banner=True)

In [ ]:
from photon_diag.palm_code import PalmSetup
from photon_diag.psen_code import PsenSetup

# Directory with calibration and other data files
home_dir = '/home/usov_i/Documents/PALM_project/'

palm = PalmSetup(home_dir=home_dir)
psen = PsenSetup(home_dir=home_dir, ttcalib=2.627)

### Calibration

In [ ]:
# Calibration plot
calib_fig = figure(plot_width=500, plot_height=300, logo=None)
circle1 = calib_fig.circle([], [], legend='eTOF1') # dummy plot to organize a figure placeholder
circle3 = calib_fig.circle([], [], legend='eTOF3', color='red')
line1 = calib_fig.line([], [], legend='eTOF1')
line3 = calib_fig.line([], [], legend='eTOF3', color='red')
calib_fig.legend.visible = False
calib_fig.xaxis.axis_label = 'Spectrometer time'
calib_fig.yaxis.axis_label = 'Photon energy, eV'
calib_plot = show(calib_fig, notebook_handle=True);

In [ ]:
# Path to folder with calibration runs
path_w = wdg.Text(
    value='etof_calib_9keV',
    description='Folder name:',
)

# Background energy
bkg_en_w = wdg.IntText(
    value=8600,
    description='Background energy (eV):',
)

# Fields with calibration constants' values
label_w = wdg.Label(
    value='Calibration constants:',
    layout=wdg.Layout(width='500px')
)

common_params = {'min': 0, 'max': 10e7, 'disabled': True, 'layout': wdg.Layout(width='150px')}
a1_w = wdg.BoundedFloatText(description='a1:', **common_params)
a3_w = wdg.BoundedFloatText(description='a3:', **common_params)
c1_w = wdg.BoundedFloatText(description='c1:', **common_params)
c3_w = wdg.BoundedFloatText(description='c3:', **common_params)

# Group elements and display
wdg.VBox([path_w, bkg_en_w])
wdg.VBox([label_w, wdg.HBox([a1_w, c1_w]), wdg.HBox([a3_w, c3_w])])

In [ ]:
def plot_fit(time, a, c):
    time_fit = np.arange(time.min(), time.max(), 0.1)  # make sure that the step is appropriate
    en_fit = (a / time_fit)**2 + c
    return time_fit, en_fit

def update(calib_res, circle, line, a_w, c_w):
    (a, c), x, y = calib_res
    x_fit, y_fit = plot_fit(x, a, c)
    circle.data_source.data = {'x': x, 'y': y}
    line.data_source.data = {'x': x_fit, 'y': y_fit}
    a_w.value = round(a, 2)
    c_w.value = round(c)

def plot_results(calib_res):
    calib_res1 = calib_res['1']
    calib_res3 = calib_res['0']
    update(calib_res1, circle1, line1, a1_w, c1_w)
    update(calib_res3, circle3, line3, a3_w, c3_w)
    calib_fig.legend.visible = True
    
    push_notebook(handle=calib_plot);

def update_button(b):
    calib_res = palm.calibrate(path_w.value, bkg_en=bkg_en_w.value, overwrite=False)
    plot_results(calib_res)

def recalib_button(b):
    calib_res = palm.calibrate(path_w.value, bkg_en=bkg_en_w.value)
    plot_results(calib_res)

# 'Update' button
b_update_w = wdg.Button(description='Update')
b_update_w.on_click(update_button)


# 'Recalibrate' button
b_recalib_w = wdg.Button(description='Recalibrate')
b_recalib_w.on_click(recalib_button)
wdg.HBox([b_update_w, b_recalib_w])

### Select file to analyse

In [ ]:
input_form = """
<div style="padding:20px;">
<input type="file" accept=".h5,.hdf5" id="select_file" name="files[]"/>
<output id="list"></output>
</div>
"""

javascript = """
<script type="text/Javascript">
  function handleFileSelect(evt) {
    var kernel = IPython.notebook.kernel;
    var files = evt.target.files;
    var output = [];
    var f = files[0]
    output.push('<li><strong>', escape(f.name), '</strong> - ',
                  f.size, ' bytes, last modified: ',
                  f.lastModifiedDate ? f.lastModifiedDate.toLocaleDateString() : 'n/a',
                  '</_Mli>');
    document.getElementById('list').innerHTML = '<ul>' + output.join('') + '</ul>';
    var command = '(lags, delays, plen), prep_data = palm.process_hdf5_file("' + f.name + '")'
    kernel.execute(command);
  }

  document.getElementById('select_file').addEventListener('change', handleFileSelect, false);
</script>
"""

def file_selector():
    from IPython.display import HTML, display
    display(HTML(input_form + javascript))
    
file_selector()

In [ ]:
# Time stage delay plot
delay_fig = figure(plot_width=500, plot_height=300, logo=None)
delay_fig.circle([], []) # dummy plot to organize a figure placeholder
delay_fig.xaxis.axis_label = 'Sacla timing tool, fs'
delay_fig.yaxis.axis_label = 'Pulse energy shift, eV'
delay_plot = show(delay_fig, notebook_handle=True);

In [ ]:
def time_delay_button(b):
    psen.load_sacla_data('392044.csv')

    # Synchronize palm and psen tags
    palm_tags = palm.tags
    sacla_tags = psen.tags

    ind = np.in1d(palm_tags, sacla_tags)
    palm_tags = palm_tags[ind]
    palm_en = delays[ind]
    ind = np.in1d(sacla_tags, palm_tags)
    sacla_tags = sacla_tags[ind]
    sacla_time = psen.data[ind]
    
    delay_fig.circle(sacla_time, palm_en, size=1)
    
    # Update time stage delay and pulse length dist figures
    push_notebook(handle=delay_plot);
    
    hist, edges = np.histogram(plen[plen!=0], bins=50)
    plength_fig.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#033649")
    push_notebook(handle=plength_plot);

# 'Time stage delay' button
b_delay_w = wdg.Button(description='Time stage delay')
b_delay_w.on_click(time_delay_button)
b_delay_w

In [ ]:
def pulse_num_change(b):
    res_arr_time = delays/0.25
    arr_time_w.value = round(res_arr_time[b.new], 2)
    pulse_length_w.value = round(plen[b.new], 2)
    
    data1 = prep_data['1']
    data0 = prep_data['0']
    pline1.data_source.data = {'x': lags, 'y': data1[b.new,:]}
    pline3.data_source.data = {'x': lags, 'y': data0[b.new,:]}
    calib_fig.legend.visible = True
    
    push_notebook(handle=xcorr_plot);

pulse_num_w = wdg.IntText(
    value=0,
    description='Pulse number:',
    disabled=False,
)

pulse_num_w.observe(pulse_num_change, names='value')
pulse_num_w

### Examples of results/intermediates/debuging visualization

In [ ]:
# Cross-corr plot
xcorr_fig = figure(plot_width=500, plot_height=300, title='Pulse arrival time', logo=None)
pline1 = xcorr_fig.line([], [], legend='eTOF1') # dummy lines to organize a figure placeholder
pline3 = xcorr_fig.line([], [], legend='eTOF3', color='red')
calib_fig.legend.visible = False
xcorr_fig.xaxis.axis_label = 'Pulse energy shift, eV'
xcorr_fig.yaxis.axis_label = 'Intensity, a.u.'
xcorr_plot = show(xcorr_fig, notebook_handle=True);

In [ ]:
arr_time_w = wdg.FloatText(
    value=0,
    description='Relative arrival time (fs):',
    disabled=True
)
arr_time_w

pulse_length_w = wdg.FloatText(
    value=0,
    description='Pulse length (fs):',
    disabled=True
)
pulse_length_w

In [ ]:
# Pulse length distribution
plength_fig = figure(plot_width=500, plot_height=300, title='Pulse lenght distribution', logo=None)
plength_fig.quad([], [], [], []) # dummy plot to organize a figure placeholder
plength_fig.xaxis.axis_label = 'Pulse length, fs'
plength_fig.yaxis.axis_label = 'Counts'
plength_plot = show(plength_fig, notebook_handle=True);

# Utility stuff

In [ ]:
tab_names = ['Calibration', 'Pulse arrival time', 'Pulse length']
children = [wdg.Text(description=name) for name in tab_names]

common_params = {'min': 0, 'max': 10e7, 'layout': wdg.Layout(width='150px')}
a1w = wdg.BoundedFloatText(description='a1:', **common_params)
a3w = wdg.BoundedFloatText(description='a3:', disabled=True, **common_params)
c1w = wdg.BoundedFloatText(description='c1:', disabled=True, **common_params)
c3w = wdg.BoundedFloatText(description='c3:', disabled=True, **common_params)
b = wdg.Button(description='Calibrate')

children[0] = wdg.HBox([wdg.VBox([a1w, a3w, b]), wdg.VBox([c1w, c3w])])

tab = wdg.Tab()
tab.children = children
for i in range(len(tab_names)):
    tab.set_title(i, tab_names[i])
tab

In [ ]:
# wdg.Dropdown(
#     options=list(map(str, range(10000))),
#     value='2',
#     description='Number:',
#     disabled=False,
# )

In [ ]:
# def change_pulse(b):
#     if b.description == "Prev":
#         if w.value != 1:
#             w.value -= 1
#     else:
#         if w.value != palm._tags.size:
#             w.value += 1

# w = wdg.BoundedIntText(min=1, max=palm._tags.size, step=1, value=1, description='Pulse index:')
# w.observe(plot_all, names='value')

# b_p = wdg.Button(description="Plot")
# b_prev = wdg.Button(description="Prev")
# b_next = wdg.Button(description="Next")
# b_prev.on_click(change_pulse)
# b_next.on_click(change_pulse)

# wdg.VBox([wdg.HBox([b_prev, b_next]), w, b])
# ---
# b_prev = wdg.Button(description="Prev")
# b_next = wdg.Button(description="Next")

# def change_pulse(b):
#     TODO: get the BoundedIntText value
#     if b.description == "Prev":
#         if curr_pulse_id != 1:
#             curr_pulse_id -= 1
#             plot_all(curr_pulse_id)
            
#     else:
#         if curr_pulse_id != palm._tags.size:
#             curr_pulse_id += 1
#             plot_all(curr_pulse_id)
        
# b_prev.on_click(change_pulse)
# b_next.on_click(change_pulse)
# wdg.HBox([b_prev, b_next])